# StrataScratch Youtube API Vid

In [1]:
!pip install google-auth
!pip install google-auth-oauthlib


In [1]:
import pandas as pd
import os
import pickle

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import googleapiclient.discovery
import googleapiclient.errors

load credentials

In [2]:
credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading Credentials From File...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

# Google's Request
from google.auth.transport.requests import Request


# If there are no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching New Tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly'
            ]
        )

        flow.run_local_server(port=8080, prompt='consent')#,authorization_prompt_message='')
        credentials = flow.credentials

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as f:
            print('Saving Credentials for Future Use...')
            pickle.dump(credentials, f)

Loading Credentials From File...
Refreshing Access Token...


In [29]:
#Get first response
def get_subs_first_response(maxResults):
        api_service_name = "youtube"
        api_version = "v3"

        youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)
        request = youtube.subscriptions().list(part="snippet,contentDetails",pageToken='CLYHEAE', maxResults=maxResults,mySubscribers=True)
        response = request.execute()
        return response, request, youtube

In [38]:
def get_subscriber_ids(df,response):
    for sub in response['items']:
        if sub['kind'] == 'youtube#subscription': 
            
            response_kind =sub['kind']
            publishedAt=sub['snippet']['publishedAt']
            channel_title =sub['snippet']['title']
            channelId = sub['snippet']['resourceId']['channelId']
            sub_channelId = sub['snippet']['channelId']        

            if response['nextPageToken'] != None:
                pageToken = response['nextPageToken']
                df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':pageToken},ignore_index=True)    
            else:
                #save data in pandas df
                df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId},ignore_index=True)
    
    return df

In [5]:
def get_subs_pagination(df,request,response,youtube):
    while 1:
        try:
            request = youtube.subscriptions().list_next(previous_request=request, previous_response=response)
            response = request.execute()
            df = get_subscriber_ids(df,response)
        
        except:
            break
    
    return df,request,response

## Main

In [42]:
#main
df = pd.DataFrame()
maxResults = 50
#get first response
response ,request, youtube = get_subs_first_response(maxResults=maxResults,)
#turn first response to df
df = get_subscriber_ids(df,response)
print(len(df))

df,request,response = get_subs_pagination(df=df,response=response ,request=request, youtube=youtube)
print(df.sub_channelId.nunique())

50
50


In [43]:
df.columns

Index(['response_kind', 'publishedAt', 'channel_title', 'channelId',
       'sub_channelId', 'pageToken'],
      dtype='object')

In [45]:
df.pageToken

0     CLYHEAA
1     CLYHEAA
2     CLYHEAA
3     CLYHEAA
4     CLYHEAA
5     CLYHEAA
6     CLYHEAA
7     CLYHEAA
8     CLYHEAA
9     CLYHEAA
10    CLYHEAA
11    CLYHEAA
12    CLYHEAA
13    CLYHEAA
14    CLYHEAA
15    CLYHEAA
16    CLYHEAA
17    CLYHEAA
18    CLYHEAA
19    CLYHEAA
20    CLYHEAA
21    CLYHEAA
22    CLYHEAA
23    CLYHEAA
24    CLYHEAA
25    CLYHEAA
26    CLYHEAA
27    CLYHEAA
28    CLYHEAA
29    CLYHEAA
30    CLYHEAA
31    CLYHEAA
32    CLYHEAA
33    CLYHEAA
34    CLYHEAA
35    CLYHEAA
36    CLYHEAA
37    CLYHEAA
38    CLYHEAA
39    CLYHEAA
40    CLYHEAA
41    CLYHEAA
42    CLYHEAA
43    CLYHEAA
44    CLYHEAA
45    CLYHEAA
46    CLYHEAA
47    CLYHEAA
48    CLYHEAA
49    CLYHEAA
Name: pageToken, dtype: object

In [41]:
print(response)

{'kind': 'youtube#SubscriptionListResponse', 'etag': 'fd-waJ7NL_f5Y2NAGsrClqCcD2k', 'prevPageToken': 'CLYHEAE', 'pageInfo': {'totalResults': 1000, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#subscription', 'etag': 'knE5VzLsTbQmjkPVzFCLP9nzHkE', 'id': 'H87wCifsGdwPu3RHY9b29dDW4Sg___skeB1Oj0jVaOg', 'snippet': {'publishedAt': '2022-02-09T09:00:30.276203Z', 'title': 'StrataScratch', 'description': 'Thanks for checking out our channel! This channel is for people beginning their careers, that are trying to learn solid fundamentals, establish themselves on the job, or get a data science job.\n\nWe cover a wide range of data science technical concepts like SQL, python, modeling, statistics, probability, business cases, etc. Our team is comprised of data scientists with years of industry experience eager to help you ace your next interview or simply improve your technical skills.\n\nIf you want hands-on technical experience, check out a platform I created, StrataScratch (www.stratascratc